In [2]:
from langchain.chat_models import BedrockChat
from langchain.schema import HumanMessage
from langchain.tools import YouTubeSearchTool
from langchain.agents import load_tools
import os
import boto3

br_client = boto3.client("bedrock-runtime", "us-east-1")

llm = BedrockChat(
    model_id="anthropic.claude-v2:1",
    client=br_client,
    model_kwargs={
        "temperature": 0
    }
)

tools = [ YouTubeSearchTool() ]

In [3]:
prompt_template = """
Human:
You are a helpful assistant, your task is help the user achive is goal.
In this environment you have access to a set of tools you can use to answer the user's question.
Use them in a smart way to fulfill our objective. Always consider the history of previous steps to ensure a continuous progress towards the objective.
When you have the answer for the user, always answer it in this forma below:
Final Answer: answer to user


You may call them like this. Only invoke one function at a time and wait for the results before invoking another function:
<function_calls>
<invoke>
<tool_name>$TOOL_NAME</tool_name>
<parameters>
<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
...
</parameters>
</invoke>
</function_calls>

Here are the tools available:
<tools>
{tools_string}
</tools>


User:
{input}

"""

In [4]:
# FUNCTION THAT CONVERT THE TOOLS FOR THE PROMPT FORMAT
from langchain_core.tools import BaseTool

from langchain_community.utils.openai_functions import (
    FunctionDescription,
    ToolDescription,
    convert_pydantic_to_openai_function
)

def format_tool_to_anthropic_function(tool: BaseTool) -> FunctionDescription:
    """Format tool into the Bedrock function API."""
    new_tool = f"""
<tool_description>
<tool_name>{tool.name}</tool_name>
<description>
{tool.description}
<parameters>
<parameter>
<name>__arg1</name>
<type>string</type>
<description>The input of the function</description>
</parameters>"""
    return new_tool

In [ ]:
# custom parser to interpret model answer
import json
from json import JSONDecodeError
from typing import List, Union

import xml.etree.ElementTree as ET

from langchain_core.agents import AgentAction, AgentActionMessageLog, AgentFinish
from langchain_core.exceptions import OutputParserException
from langchain_core.messages import (
    AIMessage,
    BaseMessage
)
from langchain_core.outputs import ChatGeneration, Generation
from langchain.agents.agent import AgentOutputParser

class AnthropicFunctionAgentOutputParser(AgentOutputParser):
    @staticmethod
    def _extract_function_call(full_text):
        # 문자열의 시자과 끝 찾기
        if "function_calls" not in full_text:
            return None
        
        start = full_text.find("<function_calls>")
        end = full_text.find("</function_calls>") + len("</function_calls>")

        # XML 문자열
        xml_string = full_text[start:end]

        # XML 문자열 분석
        root = ET.fromstring(xml_string)

        # tool name
        tool_name = root.find(".//tool_name").text

        # parameter
        parameters = {}
        for param in root.findall(".//parameters/*"):
            parameters[param.tag] = param.text

        return {
            "name": tool_name,
            "arguments": parameters
        }
    
    @property
    def _type(self) -> str:
        return "openai-functions-agent"
